In [1]:
from dsc80_utils import *

In [2]:
# Pandas Tutor setup
%reload_ext pandas_tutor
%set_pandas_tutor_options {"maxDisplayCols": 8, "nohover": True, "projectorMode": True}

# Lecture 3 – Aggregating

### Agenda

- Adding and modifying columns
- Data granularity and the `groupby` method.
- `DataFrameGroupBy` objects and aggregation.
- Other `DataFrameGroupBy` methods.
- Pivot tables using the `pivot_table` method.

You will need to code **a lot** today – make sure to pull the [course repository](https://github.com/dsc-courses/dsc80-2024-wi)

I will code a lot too, and the filled-in slides are posted online.

In [3]:
import seaborn as sns
penguins = sns.load_dataset('penguins').dropna()
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
...,...,...,...,...,...,...,...
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female
343,Gentoo,Biscoe,49.9,16.1,213.0,5400.0,Male


## Other `DataFrameGroupBy` methods

### Split-apply-combine, revisited

When we introduced the split-apply-combine pattern, the "apply" step involved **aggregation** – our final DataFrame had one row for each group.

<center><img src="imgs/image_0.png" width=40%></center>

Instead of aggregating during the apply step, we could instead perform a:

- **Transformation**, in which we perform operations to every value within each group.

- **Filtration**, in which we keep only the groups that satisfy some condition.

### Transformations

Suppose we want to convert the `'body_mass_g'` column to to z-scores (i.e. standard units):

$$z(x_i) = \frac{x_i - \text{mean of } x}{\text{SD of } x}$$

In [4]:
def z_score(x):
    return (x - x.mean()) / x.std(ddof=0)

In [5]:
z_score(penguins['body_mass_g'])

0     -0.57
1     -0.51
2     -1.19
       ... 
341    1.92
342    1.23
343    1.48
Name: body_mass_g, Length: 333, dtype: float64

### Transformations within groups

- Now, what if we wanted the z-score within each group?

- To do so, we can use the `transform` method on a `DataFrameGroupBy` object. The `transform` method takes in a function, which itself takes in a Series and returns a new Series.

- A transformation produces a DataFrame or Series of the same size – it is **not** an aggregation!

In [6]:
z_mass = (penguins
          .groupby('species')
          ['body_mass_g']
          .transform(z_score))
z_mass

0      0.10
1      0.21
2     -1.00
       ... 
341    1.32
342    0.22
343    0.62
Name: body_mass_g, Length: 333, dtype: float64

In [7]:
penguins.assign(z_mass=z_mass)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,z_mass
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,0.10
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,0.21
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,-1.00
...,...,...,...,...,...,...,...,...
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male,1.32
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female,0.22
343,Gentoo,Biscoe,49.9,16.1,213.0,5400.0,Male,0.62


In [8]:
display_df(penguins.assign(z_mass=z_mass), rows=8)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,z_mass
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,0.10
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,0.21
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,-1.00
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,-0.56
...,...,...,...,...,...,...,...,...
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female,-0.49
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male,1.32
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female,0.22
343,Gentoo,Biscoe,49.9,16.1,213.0,5400.0,Male,0.62


Note that above, penguin 340 has a larger `'body_mass_g'` than penguin 0, but a lower `'z_mass'`.
- Penguin 0 has an above average `'body_mass_g'` among `'Adelie'` penguins.
- Penguin 340 has a below average `'body_mass_g'` among `'Gentoo'` penguins. Remember from earlier that the average `'body_mass_g'` of `'Gentoo'` penguins is much higher than for other species.

In [9]:
penguins.groupby('species')['body_mass_g'].mean()

species
Adelie       3706.16
Chinstrap    3733.09
Gentoo       5092.44
Name: body_mass_g, dtype: float64

### Filtering groups

- To keep only the groups that satisfy a particular condition, use the `filter` method on a `DataFrameGroupBy` object.

- The `filter` method takes in a function, which itself takes in a DataFrame/Series and return a single Boolean. The result is a new DataFrame/Series with only the groups for which the filter function returned `True`.

For example, suppose we want only the `'species'` whose average `'bill_length_mm'` is above 39.

In [10]:
(penguins
 .groupby('species')
 .filter(lambda df: df['bill_length_mm'].mean() > 39)
)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
152,Chinstrap,Dream,46.5,17.9,192.0,3500.0,Female
153,Chinstrap,Dream,50.0,19.5,196.0,3900.0,Male
154,Chinstrap,Dream,51.3,19.2,193.0,3650.0,Male
...,...,...,...,...,...,...,...
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female
343,Gentoo,Biscoe,49.9,16.1,213.0,5400.0,Male


No more `'Adelie'`s!

Or, as another example, suppose we only want `'species'` with at least 100 penguins:

In [11]:
(penguins
 .groupby('species')
 .filter(lambda df: df.shape[0] > 100)
)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
...,...,...,...,...,...,...,...
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female
343,Gentoo,Biscoe,49.9,16.1,213.0,5400.0,Male


No more `'Chinstrap'`s!

<div class="alert alert-warning">
    <h3>Question 🤔 </h3>


Answer the following questions about grouping:

- In `.agg(fn)`, what is the input to `fn`? What is the output of `fn`?
- In `.transform(fn)`, what is the input to `fn`? What is the output of `fn`?
- In `.filter(fn)`, what is the input to `fn`? What is the output of `fn`?
</div>

### Grouping with multiple columns

When we group with multiple columns, one group is created for **every unique combination** of elements in the specified columns.

In [12]:
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
...,...,...,...,...,...,...,...
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female
343,Gentoo,Biscoe,49.9,16.1,213.0,5400.0,Male


In [13]:
species_and_island = (
    penguins
    .groupby(['species', 'island'])
    [['bill_length_mm', 'body_mass_g']]
    .mean()
)
species_and_island

bill_length_mm  body_mass_g
species   island                                
Adelie    Biscoe              38.98      3709.66
          Dream               38.52      3701.36
          Torgersen           39.04      3708.51
Chinstrap Dream               48.83      3733.09
Gentoo    Biscoe              47.57      5092.44

### Grouping and indexes

- The `groupby` method creates an index based on the specified columns.
- When grouping by multiple columns, the resulting DataFrame has a `MultiIndex`.
- Advice: When working with a `MultiIndex`, use `reset_index` or set `as_index=False` in `groupby`.

In [14]:
species_and_island

bill_length_mm  body_mass_g
species   island                                
Adelie    Biscoe              38.98      3709.66
          Dream               38.52      3701.36
          Torgersen           39.04      3708.51
Chinstrap Dream               48.83      3733.09
Gentoo    Biscoe              47.57      5092.44

In [15]:
species_and_island['body_mass_g']

species    island   
Adelie     Biscoe       3709.66
           Dream        3701.36
           Torgersen    3708.51
Chinstrap  Dream        3733.09
Gentoo     Biscoe       5092.44
Name: body_mass_g, dtype: float64

In [16]:
species_and_island.loc['Adelie']

,bill_length_mm,body_mass_g
island,,
Biscoe,38.98,3709.66
Dream,38.52,3701.36
Torgersen,39.04,3708.51


In [17]:
species_and_island.loc[('Adelie', 'Torgersen')]

bill_length_mm      39.04
body_mass_g       3708.51
Name: (Adelie, Torgersen), dtype: float64

In [18]:
species_and_island.reset_index()

,species,island,bill_length_mm,body_mass_g
0,Adelie,Biscoe,38.98,3709.66
1,Adelie,Dream,38.52,3701.36
2,Adelie,Torgersen,39.04,3708.51
3,Chinstrap,Dream,48.83,3733.09
4,Gentoo,Biscoe,47.57,5092.44


In [19]:
(penguins
 .groupby(['species', 'island'], as_index=False)
 [['bill_length_mm', 'body_mass_g']]
 .mean()
)

,species,island,bill_length_mm,body_mass_g
0,Adelie,Biscoe,38.98,3709.66
1,Adelie,Dream,38.52,3701.36
2,Adelie,Torgersen,39.04,3708.51
3,Chinstrap,Dream,48.83,3733.09
4,Gentoo,Biscoe,47.57,5092.44


<div class="alert alert-warning">
    <h3>Question 🤔 </h3>


Find the most popular `Male` and `Female` baby `Name` for each `Year` in `baby`. **Exclude** `Year`s where there were fewer than 1 million births recorded.
</div>

In [20]:
baby_path = Path('data') / 'baby.csv'
baby = pd.read_csv(baby_path)
baby

,Name,Sex,Count,Year
0,Liam,M,20456,2022
1,Noah,M,18621,2022
2,Olivia,F,16573,2022
...,...,...,...,...
2085155,Wright,M,5,1880
2085156,York,M,5,1880
2085157,Zachariah,M,5,1880


In [21]:
# Your code goes here.

## Pivot tables using the `pivot_table` method

### Pivot tables: an extension of grouping

Pivot tables are a compact way to display tables for humans to read:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Sex</th>
      <th>F</th>
      <th>M</th>
    </tr>
    <tr>
      <th>Year</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2018</th>
      <td>1698373</td>
      <td>1813377</td>
    </tr>
    <tr>
      <th>2019</th>
      <td>1675139</td>
      <td>1790682</td>
    </tr>
    <tr>
      <th>2020</th>
      <td>1612393</td>
      <td>1721588</td>
    </tr>
    <tr>
      <th>2021</th>
      <td>1635800</td>
      <td>1743913</td>
    </tr>
    <tr>
      <th>2022</th>
      <td>1628730</td>
      <td>1733166</td>
    </tr>
  </tbody>
</table>

- Notice that each value in the table is a sum over the counts, split by year and sex.
- **You can think of pivot tables as grouping using two columns, then "pivoting" one of the group labels into columns.**

### `pivot_table`

The `pivot_table` DataFrame method aggregates a DataFrame using two columns. To use it:

```py
df.pivot_table(index=index_col,
               columns=columns_col,
               values=values_col,
               aggfunc=func)
```
The resulting DataFrame will have:
- One row for every unique value in `index_col`.
- One column for every unique value in `columns_col`.
- Values determined by applying `func` on values in `values_col`.

In [22]:
last_5_years = baby.query('Year >= 2018')
last_5_years

,Name,Sex,Count,Year
0,Liam,M,20456,2022
1,Noah,M,18621,2022
2,Olivia,F,16573,2022
...,...,...,...,...
159444,Zyrie,M,5,2018
159445,Zyron,M,5,2018
159446,Zzyzx,M,5,2018


In [23]:
last_5_years.pivot_table(
    index='Year',
    columns='Sex',
    values='Count',
    aggfunc='sum',
)

Sex,F,M
Year,,
2018,1698373,1813377
2019,1675139,1790682
2020,1612393,1721588
2021,1635800,1743913
2022,1628730,1733166


In [24]:
# Look at the similarity to the snippet above!
(last_5_years
 .groupby(['Year', 'Sex'])
 [['Count']]
 .sum()
)

Count
Year Sex         
2018 F    1698373
     M    1813377
2019 F    1675139
...           ...
2021 M    1743913
2022 F    1628730
     M    1733166

[10 rows x 1 columns]

### Example

Find the number of penguins per `'island'` and `'species'`.

In [25]:
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
...,...,...,...,...,...,...,...
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female
343,Gentoo,Biscoe,49.9,16.1,213.0,5400.0,Male


In [26]:
penguins.pivot_table(
    index='species',
    columns='island',
    values='bill_length_mm', # Choice of column here doesn't actually matter!
    aggfunc='count',
)

island,Biscoe,Dream,Torgersen
species,,,
Adelie,44.0,55.0,47.0
Chinstrap,NaN,68.0,NaN
Gentoo,119.0,NaN,NaN


Note that there is a `NaN` at the intersection of `'Biscoe'` and `'Chinstrap'`, because there were no Chinstrap penguins on Biscoe Island.

We can either use the `fillna` method afterwards or the `fill_value` argument to fill in `NaN`s.

In [27]:
penguins.pivot_table(
    index='species',
    columns='island',
    values='bill_length_mm',
    aggfunc='count',
    fill_value=0,
)

island,Biscoe,Dream,Torgersen
species,,,
Adelie,44,55,47
Chinstrap,0,68,0
Gentoo,119,0,0


### Granularity, revisited

Take another look at the pivot table from the previous slide. Each row of the original `penguins` DataFrame represented a single penguin, and each column represented features of the penguins.

What is the granularity of the DataFrame below?

In [28]:
penguins.pivot_table(
    index='species',
    columns='island',
    values='bill_length_mm',
    aggfunc='count',
    fill_value=0,
)

island,Biscoe,Dream,Torgersen
species,,,
Adelie,44,55,47
Chinstrap,0,68,0
Gentoo,119,0,0


### Reshaping

- `pivot_table` reshapes DataFrames from "long" to "wide".
- Other DataFrame reshaping methods:
    - `melt`: Un-pivots a DataFrame. Very useful in data cleaning.
    - `pivot`: Like `pivot_table`, but doesn't do aggregation.
    - `stack`: Pivots multi-level columns to multi-indices.
    - `unstack`: Pivots multi-indices to columns.
    - Google and the documentation are your friends!

## Distributions

### Joint distribution

When using `aggfunc='count'`, a pivot table describes the **joint distribution** of two categorical variables. This is also called a **contingency table**.

In [29]:
counts = penguins.pivot_table(
    index='species',
    columns='sex',
    values='body_mass_g',
    aggfunc='count',
    fill_value=0
)
counts

sex,Female,Male
species,,
Adelie,73,73
Chinstrap,34,34
Gentoo,58,61


We can normalize the DataFrame by dividing by the total number of penguins. The resulting numbers can be interpreted as **probabilities** that a randomly selected penguin from the dataset belongs to a given combination of species and sex.

In [30]:
joint = counts / counts.sum().sum()
joint

sex,Female,Male
species,,
Adelie,0.22,0.22
Chinstrap,0.10,0.10
Gentoo,0.17,0.18


### Marginal probabilities

If we sum over one of the axes, we can compute **marginal probabilities**, i.e. unconditional probabilities.

In [31]:
joint

sex,Female,Male
species,,
Adelie,0.22,0.22
Chinstrap,0.10,0.10
Gentoo,0.17,0.18


In [32]:
# Recall, joint.sum(axis=0) sums across the rows,
# which computes the sum of the **columns**.
joint.sum(axis=0)

sex
Female    0.5
Male      0.5
dtype: float64

In [33]:
joint.sum(axis=1)

species
Adelie       0.44
Chinstrap    0.20
Gentoo       0.36
dtype: float64

For instance, the second Series tells us that a randomly selected penguin has a 0.36 chance of being of species `'Gentoo'`.

### Conditional probabilities

Using `counts`, how might we compute conditional probabilities like $$P(\text{species } = \text{"Adelie"} \mid \text{sex } = \text{"Female"})?$$

In [34]:
counts

sex,Female,Male
species,,
Adelie,73,73
Chinstrap,34,34
Gentoo,58,61


$$\begin{align*}
P(\text{species} = c \mid \text{sex} = x) &= \frac{\# \: (\text{species} = c \text{ and } \text{sex} = x)}{\# \: (\text{sex} = x)}
\end{align*}$$

<details>
    <summary>➡️ Click <b>here</b> to see more of a derivation.</summary>
$$\begin{align*}
P(\text{species} = c \mid \text{sex} = x) &= \frac{P(\text{species} = c \text{ and } \text{sex} = x)}{P(\text{sex = }x)} \\
&= \frac{\frac{\# \: (\text{species } = \: c \text{ and } \text{sex } = \: x)}{N}}{\frac{\# \: (\text{sex } = \: x)}{N}} \\
&= \frac{\# \: (\text{species} = c \text{ and } \text{sex} = x)}{\# \: (\text{sex} = x)}
\end{align*}$$
</details>

**Answer**: To find conditional probabilities of **`'species'` given `'sex'`**, divide by **column sums**. To find conditional probabilities of **`'sex'` given `'species'`**, divide by **row sums**.

### Conditional probabilities

To find conditional probabilities of **`'species'` given `'sex'`**, divide by **column sums**. To find conditional probabilities of **`'sex'` given `'species'`**, divide by **row sums**.

In [35]:
counts

sex,Female,Male
species,,
Adelie,73,73
Chinstrap,34,34
Gentoo,58,61


In [36]:
counts.sum(axis=0)

sex
Female    165
Male      168
dtype: int64

The conditional distribution of **`'species'` given `'sex'`** is below. Note that in this new DataFrame, the `'Female'` and `'Male'` columns each sum to 1.

In [37]:
counts / counts.sum(axis=0)

sex,Female,Male
species,,
Adelie,0.44,0.43
Chinstrap,0.21,0.20
Gentoo,0.35,0.36


For instance, the above DataFrame tells us that the probability that a randomly selected penguin is of `'species'` `'Adelie'` **given** that they are of `'sex'` `'Female'` is 0.442424.

<div class="alert alert-warning">
    <h3>Question 🤔 </h3>


Find the conditional distribution of `'sex'` given `'species'`.  

**_Hint_**: Use `.T`.
</div>

In [38]:
# Your code goes here.